# Install Taipy

To install Taipy, just `pip install` it.

In [1]:
pip install taipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pmdarima

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


# Import the packages et initialization

In [4]:
from taipy.gui import Gui, Markdown, notify
from taipy import Config
import taipy as tp

import datetime as dt

from pmdarima import auto_arima

import pandas as pd
import numpy as np

import requests

In [5]:
from pandas.core.common import SettingWithCopyWarning
import warnings

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

data_country_date = None
selected_country = None

selected_scenario = None
scenario_selector = None

scenario_name = None

result = None

selected_points = []

data_province_displayed = None

api_year = None
api_pop = None
api_country = None

# Taipy Gui Basics
## Markdown Syntax

Taipy uses the Markdown syntax to display elements. `#` creates a title, `*` puts your text in italics and `**` puts it in bold.

![](img/gui_basic.png)

In [6]:
page_md = """
# Taipy

Test **here** to put some *markdown*

Click to access the [doc](https://docs.taipy.io/en/latest/)
"""

In [9]:
page = Markdown(page_md)

gui = Gui(page=page)
gui.run(dark_mode=False,  port=8000)

 * Server starting on http://127.0.0.1:8000
 * Serving Flask app 'Taipy' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Visual elements
Create different visual elements. The syntax is always the same for each visual element.  `<|{value}|name_of_visual_element|property_1=value_of_property_1|...|>`
- Create a [slider](https://docs.taipy.io/en/latest/manuals/gui/viselements/slider/) `<|{value}|slider|>`

- Create a [date](https://docs.taipy.io/en/latest/manuals/gui/viselements/date/) `<|{value}|date|>`

- Create a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) `<|{value}|selector|lov={list_of_values}|>`


![](img/control.png)

In [10]:
slider_value = 0
date_value = None
selected_value = None

In [11]:
selector = ['Test 1', 'Test 2', 'Test 3']

control_md = """
## Controls

<|{slider_value}|slider|> <|{slider_value}|>

<|{date_value}|date|> <|{date_value}|>

<|{selected_value}|selector|lov={selector}|>  <|{selected_value}|>
"""

In [12]:
gui.stop()
page.set_content(control_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:8000
 * Serving Flask app 'Taipy' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Data Viz

A dataset gathering information on the number of deaths, confirmed cases and recovered for different regions is going to be used to create an interactive Dashboard.

In [13]:
path_to_data = "data/covid-19-all.csv"
data = pd.read_csv(path_to_data, low_memory=False)
data[-5:]

,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
1241947,Vietnam,NaN,14.058324,108.277199,1465.0,1325.0,35.0,2020-12-31
1241948,West Bank and Gaza,NaN,31.952200,35.233200,138004.0,117183.0,1400.0,2020-12-31
1241949,Yemen,NaN,15.552727,48.516388,2099.0,1394.0,610.0,2020-12-31
1241950,Zambia,NaN,-13.133897,27.849332,20725.0,18660.0,388.0,2020-12-31
1241951,Zimbabwe,NaN,-19.015438,29.154857,13867.0,11250.0,363.0,2020-12-31


In [14]:
def initialize_case_evolution(data, selected_country='France'):
    # Aggregation of the dataframe per Country/Region
    data_country_date = data.groupby(["Country/Region",'Date'])\
                            .sum()\
                            .reset_index()
    
    # a country is selected, here France by default
    data_country_date = data_country_date.loc[data_country_date['Country/Region']==selected_country]
    return data_country_date

In [15]:
data_country_date = initialize_case_evolution(data)
data_country_date.head()

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
18101,France,2020-01-24,14.6415,-56.3159,2.0,0.0,0.0
18102,France,2020-01-25,14.6415,-56.3159,3.0,0.0,0.0
18103,France,2020-01-26,14.6415,-56.3159,3.0,0.0,0.0
18104,France,2020-01-27,14.6415,-56.3159,3.0,0.0,0.0
18105,France,2020-01-28,14.6415,-56.3159,4.0,0.0,0.0


Create a [chart](https://docs.taipy.io/en/latest/manuals/gui/viselements/chart/) showing the evolution of Deaths in France (_Deaths_ for _y_ and _Date_ for _x_). The visual element (chart) has the same syntax as the other ones with specific properties (_x_, _y_, _type_ for example). Here are some [examples of charts](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/charts/bar/). The _x_ and _y_ porperties only needs the name of the dataframe columns to display.

![](img/simple_graph.png)

In [16]:
country_md = "<|{data_country_date}|chart|x=Date|y=Deaths|type=bar|>"

In [17]:
gui.stop()
page.set_content(country_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:8000
 * Serving Flask app 'Taipy' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Add new traces

- Add on the graph the number of Confirmed and Recovered cases (_Confirmed_ and _Recovered_) with the number of Deaths
- _y_ (and _x_) can be indexed this way to add more traces (`y[1]=`, `y[2]=`, `y[3]=`).

![](img/multi_traces.png)

In [18]:
country_md = "<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|>"

In [19]:
gui.stop()
page.set_content(country_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:8000
 * Serving Flask app 'Taipy' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Style the graph with personnalized properties
The _layout_ dictionnary specifies how bars should be displayed. They would be 'stacked'.

The _options_ dictionary will change the opacity of the unselected markers.

These are Plotly properties.

![](img/stack_chart.png)

In [20]:
layout = {'barmode':'stack'}
options = {"unselected":{"marker":{"opacity":0.5}}}
country_md = "<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>"

In [21]:
gui.stop()
page.set_content(country_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:8000
 * Serving Flask app 'Taipy' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Add texts that sums up the data

Use the [text](https://docs.taipy.io/en/latest/manuals/gui/viselements/text/) visual element.

- Add the total number of Deaths (last line of _data_country_date_)
- Add the total number of Recovered (last line of _data_country_date_)
- Add the total number of Confirmed (last line of _data_country_date_)


In [22]:
data_country_date

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
18101,France,2020-01-24,14.641500,-56.315900,2.0,0.0,0.0
18102,France,2020-01-25,14.641500,-56.315900,3.0,0.0,0.0
18103,France,2020-01-26,14.641500,-56.315900,3.0,0.0,0.0
18104,France,2020-01-27,14.641500,-56.315900,3.0,0.0,0.0
18105,France,2020-01-28,14.641500,-56.315900,4.0,0.0,0.0
...,...,...,...,...,...,...,...
18439,France,2020-12-27,91.463495,-238.646914,2616510.0,195861.0,62867.0
18440,France,2020-12-28,91.463495,-238.646914,2619616.0,196642.0,63235.0
18441,France,2020-12-29,91.463495,-238.646914,2631110.0,197726.0,64204.0
18442,France,2020-12-30,91.463495,-238.646914,2657624.0,198966.0,64508.0


This is how we can get the total number of Deaths from the daatset for France.

In [23]:
data_country_date.iloc[-1, 6] # gives the number of deaths for France (5 is for recovered and 4 is confirmed)

64759.0

Use the [text](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/text/) visual element. Note that between `{}`, any Python variable can be put but also any Python code.

![](img/control_text.png)

In [24]:
country_md = """
## Deaths <|{data_country_date.iloc[-1, 6]}|text|>

## Recovered <|{data_country_date.iloc[-1, 5]}|text|>

## Confirmed <|{data_country_date.iloc[-1, 4]}|text|>

<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [25]:
gui.stop()
page.set_content(country_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:8000
 * Serving Flask app 'Taipy' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Local _on_change_

- Add a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) with `dropdown=True` containing the name of all the _Country/region_
- Give to the _on_change_ selector property the name of the _on_change_country_ function. This function will be called when the selector will be used.
- This function has a 'state' parameter and has to be completed. When the selector is used, this function is called with the _state_ argument. It contains all the Gui variables; 'state.data_country_date' is then the dataframe used in the Gui.

![](img/on_change_local.png)

In [26]:
selector_country = list(data['Country/Region'].astype(str).unique())
selected_country = 'France'

country_md = """
<|{selected_country}|selector|lov={selector_country}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{data_country_date.iloc[-1, 6]}|>

## Recovered <|{data_country_date.iloc[-1, 5]}|>

## Confirmed <|{data_country_date.iloc[-1, 4]}|>

<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [27]:
def on_change_country(state):
    # state contains all the Gui variables and this is through this state variable that we can update the Gui
    # state.selected_country, state.data_country_date, ...
    # update data_country_date with the right country (use initialize_case_evolution)
    print("Chosen country: ", state.selected_country)
    state.data_country_date = initialize_case_evolution(data, state.selected_country)

In [28]:
gui.stop()
page.set_content(country_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:8000
 * Serving Flask app 'Taipy' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Layout

Use the [layout](https://docs.taipy.io/en/latest/manuals/gui/viselements/layout/) block to change the page structure. This block creates invisible columns to put text/visual elements in.

Syntax :
```
<|layout|columns=1 1 1 ...|
(first column)

(in second column)

(third column)
(again, third column)

(...)
|>
```

In [29]:
final_country_md = """
<|layout|columns=1 1 1 1|
<|{selected_country}|selector|lov={selector_country}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{data_country_date.iloc[-1, 6]}|>

## Recovered <|{data_country_date.iloc[-1, 5]}|>

## Confirmed <|{data_country_date.iloc[-1, 4]}|>
|>

<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [30]:
gui.stop()
page.set_content(final_country_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:8000
 * Serving Flask app 'Taipy' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


![](img/layout.png)

# Map

In [31]:
def initialize_map(data):
    data['Province/State'] = data['Province/State'].fillna(data["Country/Region"])
    data_province = data.groupby(["Country/Region",
                                  'Province/State',
                                  'Longitude',
                                  'Latitude'])\
                         .max()

    data_province_displayed = data_province[data_province['Deaths']>10].reset_index()

    data_province_displayed['Size'] = np.sqrt(data_province_displayed.loc[:,'Deaths']/data_province_displayed.loc[:,'Deaths'].max())*80 + 3
    data_province_displayed['Text'] = data_province_displayed.loc[:,'Deaths'].astype(str) + ' deaths </br> ' + data_province_displayed.loc[:,'Province/State']
    return data_province_displayed

In [32]:
data_province_displayed = initialize_map(data)
data_province_displayed.head()

,Country/Region,Province/State,Longitude,Latitude,Confirmed,Recovered,Deaths,Date,Size,Text
0,Afghanistan,Afghanistan,67.709953,33.93911,51526.0,41727.0,2191.0,2020-12-31,17.771247,2191.0 deaths </br> Afghanistan
1,Albania,Albania,20.168300,41.15330,58316.0,33634.0,1181.0,2020-12-31,13.844784,1181.0 deaths </br> Albania
2,Algeria,Algeria,1.659600,28.03390,99610.0,67127.0,2756.0,2020-12-31,19.566684,2756.0 deaths </br> Algeria
3,Andorra,Andorra,1.521800,42.50630,8049.0,7432.0,84.0,2020-12-31,5.892249,84.0 deaths </br> Andorra
4,Angola,Angola,17.873900,-11.20270,17553.0,11044.0,405.0,2020-12-31,9.350728,405.0 deaths </br> Angola


Properties to style the map
- marker color corresponds to the number of Deaths (column _Deaths_)
- marker sizes corresponds to the size in _Size_ column which is created from the number of Deaths

layout_map permet defined the initial zoom and position of the map


In [33]:
marker_map = {"color":"Deaths", "size": "Size", "showscale":True, "colorscale":"Viridis"}
layout_map = {
            "dragmode": "zoom",
            "mapbox": { "style": "open-street-map", "center": { "lat": 38, "lon": -90 }, "zoom": 3}
            }

We give to Plotly:
- a map type
- the name of the latitude column
- the name of the longitude column
- properties: on the size and color of the markers
- the name of the column for the text of the points

In [34]:
map_md = """
<|{data_province_displayed}|chart|type=scattermapbox|selected={selected_points}|lat=Latitude|lon=Longitude|marker={marker_map}|layout={layout_map}|text=Text|mode=markers|height=800px|options={options}|>
"""

In [35]:
gui.stop()
page.set_content(map_md)
gui.run()

Gui server has been stopped


C:\Users\jacta\anaconda3\lib\site-packages\taipy\gui\gui.py:337: UserWarning: Decoding Message has failed: {'type': 'RU', 'name': '', 'payload': {'names': ['_TpL_tpec_TpExPr_selector_country_TPMDL_0', '_TpLv_tpec_TpExPr_selected_country_TPMDL_0']}, 'propagate': True, 'client_id': '20220829180417100218-0.8424481706569947', 'module_context': ''}
'types.SimpleNamespace' object has no attribute '_TpL_tpec_TpExPr_selector_country_TPMDL_0'
  warnings.warn(f"Decoding Message has failed: {message}\n{e}")
C:\Users\jacta\anaconda3\lib\site-packages\taipy\gui\gui.py:337: UserWarning: Decoding Message has failed: {'type': 'DU', 'name': '_TpD_tpec_TpExPr_data_country_date_TPMDL_0', 'payload': {'pagekey': 'Deaths-Recovered-Date-Confirmed', 'columns': ['Deaths', 'Recovered', 'Date', 'Confirmed'], 'alldata': True}, 'propagate': True, 'client_id': '20220829180417100218-0.8424481706569947', 'module_context': ''}
'types.SimpleNamespace' object has no attribute '_TpD_tpec_TpExPr_data_country_date_TPMDL_0'

 * Server starting on http://127.0.0.1:8000
 * Serving Flask app 'Taipy' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


![](img/carte.png)

# Part and the _render_ property
- Create a [toggle](https://docs.taipy.io/en/latest/manuals/gui/viselements/toggle/) (works the same as a selector) with a lov of 'Map' an 'Country'
- Create two part blocks that renders or not depending on the value of the toggle
    - To do this, use the fact that in the _render_ property of the part block, Python code can be inserted in `{}`

In [36]:
representation_selector = ['Map', 'Country']
selected_representation = representation_selector[0]

In [37]:
main_page = """
<|{selected_representation}|toggle|lov={representation_selector}|>

<|part|render={selected_representation == "Country"}|
"""+final_country_md+"""
|>

<|part|render={selected_representation == "Map"}|
"""+map_md+"""
|>
""" 

In [38]:
gui.stop()
page.set_content(main_page)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:8000
 * Serving Flask app 'Taipy' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


![](img/part_render.png)

# Taipy Core
Here are the functions that we are going to use to predict the number of Deaths for a country.
We will:
- preprocess the data (_preprocess_),
- create a training and testing database (_make_train_test_data_),
- train a model (_train_model_),
- generate predictions (_forecast_),
- generate a dataframe with the historical data and the predictions (_result_)

![](img/all_architecture.svg)

In [39]:
def preprocess(initial_data, country):
    data = initial_data.groupby(["Country/Region",'Date'])\
                       .sum()\
                       .dropna()\
                       .reset_index()

    preprocess_data = data.loc[data['Country/Region']==country].reset_index(drop=True)
    return preprocess_data
    
    
def make_train_test_data(preprocess_data):
    perc_test = 0.3
    nb_to_predict = int(perc_test*len(preprocess_data))
    train_data = preprocess_data[:-nb_to_predict]
    return train_data, nb_to_predict


def train_model(train_data):
    model = auto_arima(train_data['Deaths'],
                       start_p=1, start_q=1,
                       max_p=5, max_q=5,
                       start_P=0, seasonal=False,
                       d=1, D=1, trace=True,
                       error_action='ignore',  
                       suppress_warnings=True)
    model.fit(train_data['Deaths'])
    return model


def forecast(model, nb_to_predict):
    predictions = model.predict(n_periods=nb_to_predict)
    return predictions


def result(preprocess_data, predictions, nb_to_predict):
    pred_series = pd.Series([np.NaN]*(len(preprocess_data)-nb_to_predict) + list(predictions), name='Predictions')
    return pd.concat([preprocess_data, pred_series], axis=1)

First we must define the Data Nodes then the tasks (associated to the Python function). Furthermore, we gather these tasks into different pipelines and these pipelines into a scenario.

A Data Nod needs a **unique id**. If needed, the storage type can be changed for CSV and SQL. Other parameters are then needed.

### Data Nodes and Task for preprocess

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [40]:
initial_data_cfg = Config.configure_data_node(id="initial_data",
                                              storage_type="csv",
                                              path=path_to_data,
                                              cacheable=True,
                                              validity_period=dt.timedelta(days=5))

country_cfg = Config.configure_data_node(id="country", default_data="France",
                                         cacheable=True, validity_period=dt.timedelta(days=5))

preprocess_data_cfg =  Config.configure_data_node(id="preprocess_data",
                                                  cacheable=True, validity_period=dt.timedelta(days=5))


task_preprocess_cfg = Config.configure_task(id="task_preprocess_data",
                                           function=preprocess,
                                           input=[initial_data_cfg, country_cfg],
                                           output=preprocess_data_cfg)

### Data Nodes and Task for make_train_test_data

<img src="img/make_train_test_data.svg" alt="drawing" width="500"/>

In [41]:
train_data_cfg =  Config.configure_data_node(id="train_data", cacheable=True, validity_period=dt.timedelta(days=5))

nb_to_predict_cfg = Config.configure_data_node(id="nb_to_predict",
                                               cacheable=True, validity_period=dt.timedelta(days=5))

task_train_test_cfg = Config.configure_task(id="task_make_train_test_data",
                                           function=make_train_test_data,
                                           input=preprocess_data_cfg,
                                           output=[train_data_cfg, nb_to_predict_cfg])

### Data Nodes and Task for train_model

<img src="img/train_model.svg" alt="drawing" width="500"/>

In [42]:
model_cfg = Config.configure_data_node(id="model", cacheable=True, validity_period=dt.timedelta(days=5))

task_train_cfg = Config.configure_task(id="task_train",
                                      function=train_model,
                                      input=train_data_cfg,
                                      output=model_cfg)

### Data Nodes and Task for forecast

<img src="img/forecast.svg" alt="drawing" width="500"/>

In [43]:
predictions_cfg = Config.configure_data_node(id="predictions")

task_forecast_cfg = Config.configure_task(id="task_forecast",
                                      function=forecast,
                                      input=[model_cfg, nb_to_predict_cfg],
                                      output=predictions_cfg)

### Data Nodes and Task for result

<img src="img/result.svg" alt="drawing" width="500"/>

In [44]:
result_cfg = Config.configure_data_node(id="result")

task_result_cfg = Config.configure_task(id="task_result",
                                      function=result,
                                      input=[preprocess_data_cfg, predictions_cfg, nb_to_predict_cfg],
                                      output=result_cfg)

## [Configuration of pipelines](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_pipeline)

In [45]:
pipeline_preprocessing_cfg = Config.configure_pipeline(id="pipeline_preprocessing",
                                                       task_configs=[task_preprocess_cfg, task_train_test_cfg])

pipeline_train_cfg = Config.configure_pipeline(id="pipeline_train",
                                               task_configs=[task_train_cfg])

pipeline_forecast_cfg = Config.configure_pipeline(id="pipeline_forecast",
                                                       task_configs=[task_forecast_cfg, task_result_cfg])

## [Configuration of scénario](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_scenario)

In [46]:
scenario_cfg = Config.configure_scenario(id='scenario', pipeline_configs=[pipeline_preprocessing_cfg,
                                                                         pipeline_train_cfg,
                                                                         pipeline_forecast_cfg])

## Creation and submit of scenario

In [47]:
scenario = tp.create_scenario(scenario_cfg, name='First Scenario')
tp.submit(scenario)

C:\Users\jacta\anaconda3\lib\site-packages\taipy\core\data\csv.py:145: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-08-29 18:04:48,986][Taipy][INFO] job JOB_task_preprocess_data_ee6e5561-c41c-44d1-aec9-866116e1c749 is completed.
[2022-08-29 18:04:49,128][Taipy][INFO] job JOB_task_make_train_test_data_be33ce04-571a-40db-a317-6b01a219c256 is completed.
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2994.124, Time=0.18 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3349.636, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3045.305, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3200.566, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3402.421, Time=0.01 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2996.090, Time=0.18 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=2996.081, Time=0.15 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3162.541, Time=0.13 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3014.299, Time=0.04 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=2996.306, Time=0.16 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2992.816, Ti

In [48]:
scenario.model.read()

ARIMA(order=(1, 1, 1), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [49]:
scenario.result.read()

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths,Predictions
0,France,2020-01-24,14.641500,-56.315900,2.0,0.0,0.0,NaN
1,France,2020-01-25,14.641500,-56.315900,3.0,0.0,0.0,NaN
2,France,2020-01-26,14.641500,-56.315900,3.0,0.0,0.0,NaN
3,France,2020-01-27,14.641500,-56.315900,3.0,0.0,0.0,NaN
4,France,2020-01-28,14.641500,-56.315900,4.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
338,France,2020-12-27,91.463495,-238.646914,2616510.0,195861.0,62867.0,32931.922617
339,France,2020-12-28,91.463495,-238.646914,2619616.0,196642.0,63235.0,32936.896572
340,France,2020-12-29,91.463495,-238.646914,2631110.0,197726.0,64204.0,32941.767204
341,France,2020-12-30,91.463495,-238.646914,2657624.0,198966.0,64508.0,32946.536660


In [50]:
scenario.predictions.read()

array([31336.91486027, 31375.02135569, 31412.33627815, 31448.87607071,
       31484.65683488, 31519.6943377 , 31554.00401867, 31587.60099658,
       31620.50007616, 31652.71575462, 31684.262228  , 31715.15339748,
       31745.40287544, 31775.02399153, 31804.02979849, 31832.43307792,
       31860.24634591, 31887.48185857, 31914.15161741, 31940.26737463,
       31965.84063831, 31990.8826775 , 32015.40452712, 32039.4169929 ,
       32062.93065608, 32085.95587812, 32108.50280525, 32130.5813729 ,
       32152.20131016, 32173.37214398, 32194.10320345, 32214.40362383,
       32234.28235065, 32253.7481436 , 32272.80958041, 32291.47506062,
       32309.75280931, 32327.6508807 , 32345.17716167, 32362.33937532,
       32379.14508427, 32395.60169408, 32411.71645647, 32427.49647251,
       32442.94869576, 32458.07993537, 32472.896859  , 32487.40599583,
       32501.61373942, 32515.52635049, 32529.14995974, 32542.4905705 ,
       32555.55406139, 32568.34618894, 32580.87259007, 32593.13878463,
      

## Caching
Some job are skipped because no change has been done to the "input" Data Nodes.

In [51]:
tp.submit(scenario)

[2022-08-29 18:04:51,121][Taipy][INFO] job JOB_task_preprocess_data_b138ff25-f82b-40d3-a34c-19e3f370e36c is skipped.
[2022-08-29 18:04:51,236][Taipy][INFO] job JOB_task_make_train_test_data_a1dfdaf0-ee38-4994-8e0f-12da983e4937 is skipped.
[2022-08-29 18:04:51,360][Taipy][INFO] job JOB_task_train_b6fa11df-a673-4610-aeb7-193983ea49b4 is skipped.
[2022-08-29 18:04:51,488][Taipy][INFO] job JOB_task_forecast_47a6e187-c6c1-4e67-a1f9-5540a2a9d030 is completed.
[2022-08-29 18:04:51,599][Taipy][INFO] job JOB_task_result_8adc6c29-a9eb-42f7-9b2e-58bdd9d3589e is completed.


## Write in data nodes

To write a data node:

`<Data Node>.write(new_value)`

In [52]:
scenario.country.write('US')
tp.submit(scenario)
scenario.result.read()

C:\Users\jacta\anaconda3\lib\site-packages\taipy\core\data\csv.py:145: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-08-29 18:04:52,985][Taipy][INFO] job JOB_task_preprocess_data_5181ddc3-d475-4a2a-87fa-c9681fda9c22 is completed.
[2022-08-29 18:04:53,097][Taipy][INFO] job JOB_task_make_train_test_data_254a94f2-9e4e-4d86-9ca9-b0a4b0710dce is completed.
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3565.937, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3869.494, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3564.625, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3712.670, Time=0.07 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=4064.717, Time=0.01 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3565.875, Time=0.03 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3567.852, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=3573.780, Time=0.01 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0] intercept
Total fit time: 0.249 seconds
[2022-08-29 18:04:53,459][Taipy][INFO] job JOB_task_train_862fbf6a-8a57-42ae-ba59-c960636b3cbf is completed.


,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths,Predictions
0,US,2020-01-22,47.140037,-120.791360,1.0,0.0,0.0,NaN
1,US,2020-01-23,47.140037,-120.791360,1.0,0.0,0.0,NaN
2,US,2020-01-24,85.345498,-210.450080,2.0,0.0,0.0,NaN
3,US,2020-01-25,87.000686,-209.922210,2.0,0.0,0.0,NaN
4,US,2020-01-26,158.375485,-442.264230,5.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
340,US,2020-12-27,124119.377356,-298994.802493,19222064.0,0.0,334533.0,281684.136115
341,US,2020-12-28,124119.377356,-298994.802493,19396237.0,0.0,336438.0,282520.640662
342,US,2020-12-29,124119.377356,-298994.802493,19595117.0,0.0,340061.0,283357.145209
343,US,2020-12-30,124119.377356,-298994.802493,19827133.0,0.0,343783.0,284193.649757


## Simple framework

In [53]:
scenario = tp.create_scenario(scenario_cfg, name='Second Scenario')
tp.submit(scenario)

C:\Users\jacta\anaconda3\lib\site-packages\taipy\core\data\csv.py:145: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-08-29 18:04:55,132][Taipy][INFO] job JOB_task_preprocess_data_da57847c-8ddf-4377-a02a-abfdcd9b08c7 is completed.
[2022-08-29 18:04:55,260][Taipy][INFO] job JOB_task_make_train_test_data_476a7862-d95c-4c0c-b202-b3621fb04331 is completed.
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2994.124, Time=0.11 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3349.636, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3045.305, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3200.566, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3402.421, Time=0.01 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2996.090, Time=0.19 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=2996.081, Time=0.15 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3162.541, Time=0.11 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3014.299, Time=0.03 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=2996.306, Time=0.16 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2992.816, Ti

In [54]:
[s.model.read() for s in tp.get_scenarios()]

[ARIMA(order=(1, 1, 0), scoring_args={}, suppress_warnings=True),
 ARIMA(order=(1, 1, 1), scoring_args={}, suppress_warnings=True,
       with_intercept=False)]

## Create a Gui for the backend
_scenario_selector_ lets you choose a scenario and display its results.

In [55]:
scenario_selector = [(s.id, s.name) for s in tp.get_scenarios()]
selected_scenario = scenario.id
print(scenario_selector,'\n', selected_scenario)

[('SCENARIO_scenario_5b40be93-759c-4705-a1da-9db0962630d2', 'First Scenario'), ('SCENARIO_scenario_c76146a7-0f6e-4181-ab64-d3520565cf18', 'Second Scenario')] 
 SCENARIO_scenario_c76146a7-0f6e-4181-ab64-d3520565cf18


In [56]:
result = scenario.result.read()
result

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths,Predictions
0,France,2020-01-24,14.641500,-56.315900,2.0,0.0,0.0,NaN
1,France,2020-01-25,14.641500,-56.315900,3.0,0.0,0.0,NaN
2,France,2020-01-26,14.641500,-56.315900,3.0,0.0,0.0,NaN
3,France,2020-01-27,14.641500,-56.315900,3.0,0.0,0.0,NaN
4,France,2020-01-28,14.641500,-56.315900,4.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
338,France,2020-12-27,91.463495,-238.646914,2616510.0,195861.0,62867.0,32931.922617
339,France,2020-12-28,91.463495,-238.646914,2619616.0,196642.0,63235.0,32936.896572
340,France,2020-12-29,91.463495,-238.646914,2631110.0,197726.0,64204.0,32941.767204
341,France,2020-12-30,91.463495,-238.646914,2657624.0,198966.0,64508.0,32946.536660


**Tips** : the _value_by_id_ property if set to True for a selected will make _selected_scenario_ directly refer to the first element of the tupple (here the id)

![](img/predictions.png)

In [57]:
prediction_md = """
<|layout|columns=1 5 1 3|
<|{scenario_name}|input|label=Name|>

<br/>
<|Create|button|on_action=create_new_scenario|>

<|{selected_country}|selector|lov={selector_country}|dropdown|on_change=on_change_country|label=Country|>

<br/>
<|Submit|button|on_action=submit_scenario|>

<|{selected_scenario}|selector|lov={scenario_selector}|on_change=actualize_graph|dropdown|value_by_id|label=Scenario|>
|>

<|{result}|chart|x=Date|y[1]=Deaths|type[1]=bar|y[2]=Predictions|>
"""

In [58]:
def create_new_scenario(state):
    scenario = tp.create_scenario(scenario_cfg, name=state.scenario_name)
    state.scenario_selector += [(scenario.id, scenario.name)]

In [59]:
def actualize_graph(state):
    # 1) update the result dataframe
    # 2) change selected_country with the predicted country of the scenario
    scenario = tp.get(state.selected_scenario)
    state.result = scenario.result.read()
    state.selected_country = scenario.country.read()

In [60]:
def submit_scenario(state):
    # 1) get the selected scenario
    # 2) write in country Data Node, the selected country
    # 3) submit the scenario
    # 4) actualize le graph avec actualize_graph
    scenario = tp.get(state.selected_scenario)
    scenario.country.write(state.selected_country)
    tp.submit(scenario)
    actualize_graph(state)

In [61]:
gui.stop()
page.set_content(prediction_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:8000
 * Serving Flask app 'Taipy' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


# Multi-pages and Taipy Rest

To create a multi-pages app, we only need a dictionary with names as the keys and the Markdowns as the values.

The _navbar_ control (<|navbar|>) has a default behaviour. It redirects to the different pages of the app automatically. Other solutions exists.

![](img/multi_pages.png)

In [62]:
navbar_md = "<center>\n<|navbar|>\n</center>"

pages = {
    "Map":navbar_md+map_md,
    "Country":navbar_md+final_country_md,
    "Predictions":navbar_md+prediction_md
}

rest = tp.Rest()

gui_multi_pages = Gui(pages=pages)
tp.run(gui_multi_pages, rest, dark_mode=False, port=5006)

 * Server starting on http://127.0.0.1:5006
 * Serving Flask app 'taipy.rest.app' (lazy loading)
 * Environment: None
 * Debug mode: off
Chosen country:  Mexico
Chosen country:  China


C:\Users\jacta\anaconda3\lib\site-packages\taipy\core\data\csv.py:145: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-08-29 18:05:51,775][Taipy][INFO] job JOB_task_preprocess_data_8c501c82-5289-4146-8658-213234aa12cb is completed.
[2022-08-29 18:05:51,911][Taipy][INFO] job JOB_task_make_train_test_data_237c9f10-1443-4d21-8150-8ce6aa607212 is completed.
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2847.403, Time=0.14 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2852.683, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2853.352, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2853.560, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2862.023, Time=0.01 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2849.161, Time=0.22 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=2849.159, Time=0.21 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=2853.768, Time=0.11 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=2853.148, Time=0.05 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.22 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2847.356, Time=0.